# A Case Study: The Effect of Gun Ownership on Gun-Homicide Rates

We consider the problem of estimating the effect of gun
ownership on the homicide rate. For this purpose, we estimate the following partially
linear model

$$
 Y_{j,t} = \beta D_{j,(t-1)} + g(Z_{j,t}) + \epsilon_{j,t}.
$$

## Data

$Y_{j,t}$ is log homicide rate in county $j$ at time $t$, $D_{j, t-1}$ is log  fraction of suicides committed with a firearm in county $j$ at time $t-1$, which we use as a proxy for gun ownership,  and  $Z_{j,t}$ is a set of demographic and economic characteristics of county $j$ at time $t$. The parameter $\beta$ is the effect of gun ownership on the
homicide rates, controlling for county-level demographic and economic characteristics. 

The sample covers 195 large United States counties between the years 1980 through 1999, giving us 3900 observations.

In [150]:
using Pkg
Pkg.add("CSV"), using CSV
Pkg.add("DataFrames"), using DataFrames
Pkg.add("StatsModels"), using StatsModels
Pkg.add("GLM"), using GLM
Pkg.add("Random"), using Random

   Resolving package versions...
  No Changes to `C:\Users\PC\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\PC\.julia\environments\v1.6\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\PC\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\PC\.julia\environments\v1.6\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\PC\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\PC\.julia\environments\v1.6\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\PC\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\PC\.julia\environments\v1.6\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\PC\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\PC\.julia\environments\v1.6\Manifest.toml`


(nothing, nothing)

In [2]:
data = CSV.File("../data/gun_clean.csv") |> DataFrame;
println("Number of rows: ",size(data,1))
println("Number of columns: ",size(data,2))

Number of rows: 3900
Number of columns: 415


### Preprocessing

To account for heterogeneity across counties and time trends in  all variables, we remove from them county-specific and time-specific effects in the following preprocessing.

In [3]:
function varlist(df = nothing , type_dataframe = ["numeric","categorical","string"], pattern=String , exclude =  nothing)

    varrs = []
    if "numeric" in type_dataframe
        append!(varrs, [i for i in names(data) if eltype(eachcol(data)[i]) <: Number])    
    end
    if "categorical" in type_dataframe
        append!(varrs,[i for i in names(data) if eltype(eachcol(data)[i]) <: CategoricalVector])
    end
    if "string" in type_dataframe
        append!(varrs,[i for i in names(data) if eltype(eachcol(data)[i]) <: String])
    end
    varrs[(!varrs in exclude) & varrs[findall(x->contains(x,pattern),names(data))]]
end

varlist (generic function with 5 methods)

In [4]:
fixed = filter(x->contains(x,"X_Jfips"),names(data));
year = filter(x->contains(x,"X_Tyear"),names(data));

In [5]:
size(year)

(21,)

In [6]:
census = []
census_var = ["AGE", "BN", "BP", "BZ", "ED", "EL","HI", "HS", "INC", "LF", "LN", "PI", "PO", "PP", "PV", "SPR", "VS"]
for i in 1:size(census_var,1) 
    append!(census,filter(x->contains(x,census_var[i]),names(data)))
end

In [7]:
d = ["logfssl"]
y = ["logghomr"]
X1 = ["logrobr", "logburg", "burg_missing", "robrate_missing"]
X2 = ["newblack", "newfhh", "newmove", "newdens", "newmal"]

variable = [y, d,X1, X2, census]
varlis = []
for i in variable
    append!(varlis,i)
end


In [8]:
example = DataFrame(CountyCode = data[:,"CountyCode"]);
rdata = DataFrame(CountyCode = data[:,"CountyCode"]);

for i in 1:size(varlis,1)
    rdata[!,varlis[i]]= residuals(lm(term(Symbol(varlis[i])) ~ sum(term.(Symbol.(year))) + sum(term.(Symbol.(fixed))), data))
end

In [9]:
size(rdata)

(3900, 198)

Now, we can construct the treatment variable, the outcome variable and the matrix $Z$ that includes the control variables.

In [10]:
# treatment variable

D = rdata[!,d]
# Outcome variable
Y = rdata[!,y];

# Construct matrix Z
Z = rdata[!, varlis[3:end]]

size(Z)

(3900, 195)

We have in total 195 control variables. The control variables $Z_{j,t}$ are from the U.S. Census Bureau and  contain demographic and economic characteristics of the counties such as  the age distribution, the income distribution, crime rates, federal spending, home ownership rates, house prices, educational attainment, voting paterns, employment statistics, and migration rates. 

In [12]:
clu = select(rdata,:CountyCode)
data = hcat(Y,D,Z,clu);

In [13]:
size(data)

(3900, 198)

## The effect of gun ownership


### OLS

After preprocessing the data, we first look at simple regression of $Y_{j,t}$ on $D_{j,t-1}$ without controls as a baseline model.

In [14]:
Pkg.add("FixedEffectModels"), using FixedEffectModels

   Resolving package versions...
  No Changes to `C:\Users\PC\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\PC\.julia\environments\v1.6\Manifest.toml`


(nothing, nothing)

In [130]:
baseline_ols = reg(data, @formula(logghomr ~ 1 + logfssl +fe(CountyCode)));
est_baseline = coef(baseline_ols) ;
println(confint(baseline_ols))
println(est_baseline)


[0.16710155204187516 0.3975073812245494]
[0.2823044666332123]


The point estimate is $0.282$ with the confidence interval ranging from 0.167 to 0.397. This
suggests that increases in gun ownership rates are related to gun homicide rates - if gun ownership increases by 1% relative
to a trend then the predicted gun homicide rate goes up by 0.28%, without controlling for counties' characteristics.

Since our goal is to estimate the effect of gun ownership after controlling for a rich set county characteristics we next include the controls. First, we estimate the model by ols and then by an array of the modern regression methods using the double machine learning approach.

In [145]:
confint(control_ols)[1,1:2]

2-element Vector{Float64}:
 0.07713994894072317
 0.30415014815530284

In [149]:
control_ols= reg(data,term(:logghomr) ~  term(:logfssl) + sum(term.(Symbol.(names(Z)))) + fe(:CountyCode))
est_ols = coef(control_ols)[1]
println(confint(control_ols)[1,1:2])
println(est_ols[1])


[0.07713994894072317, 0.30415014815530284]
0.190645048548013


# DML algorithm

Here we perform inference of the predictive coefficient $\beta$ in our partially linear statistical model, 

$$
Y = D\beta + g(Z) + \epsilon, \quad E (\epsilon | D, Z) = 0,
$$

using the **double machine learning** approach. 

For $\tilde Y = Y- E(Y|Z)$ and $\tilde D= D- E(D|Z)$, we can write
$$
\tilde Y = \alpha \tilde D + \epsilon, \quad E (\epsilon |\tilde D) =0.
$$

Using cross-fitting, we employ modern regression methods
to build estimators $\hat \ell(Z)$ and $\hat m(Z)$ of $\ell(Z):=E(Y|Z)$ and $m(Z):=E(D|Z)$ to obtain the estimates of the residualized quantities:

$$
\tilde Y_i = Y_i  - \hat \ell (Z_i),   \quad \tilde D_i = D_i - \hat m(Z_i), \quad \text{ for each } i = 1,\dots,n.
$$

Finally, using ordinary least squares of $\tilde Y_i$ on $\tilde D_i$, we obtain the 
estimate of $\beta$.

In [187]:
Pkg.add("MLDataUtils"), using  MLDataUtils
Pkg.add("MLBase"), using  MLBase 


   Resolving package versions...
  No Changes to `C:\Users\PC\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\PC\.julia\environments\v1.6\Manifest.toml`
Precompiling project...
  ✓ Distances
  ✓ LinearRegression
  ✓ FilePathsBase
  ✓ ChainRulesCore
  ✓ WeakRefStrings
  ✓ PersistenceDiagramsBase
  ✓ JSON3
  ✓ Loess
  ✓ DataStructures
  ✓ AbstractFFTs
  ✓ ChangesOfVariables
  ✓ NearestNeighbors
  ✓ JLSO
  ✓ JSON
  ✓ HDF5
  ✓ TimeZones
  ✓ SortingAlgorithms
  ✓ QuadGK
  ✓ NamedArrays
  ✓ NNlib
  ✓ Interpolations
  ✓ Pickle
  ✓ MLJOpenML
  ✓ TableShowUtils
  ✓ LogExpFunctions
  ✓ CSV
  ✓ Conda
  ✓ GPUArrays
  ✓ AssetRegistry
  ✓ CategoricalArrays
  ✓ ChainRules
  ✓ GPUCompiler
  ✓ FFTW
  ✓ MAT
  ✓ DataTables
  ✓ Intervals
  ✓ PlotlyBase
  ✓ StatsBase
  ✓ GR
  ✓ MLJScientificTypes
  ✓ IJulia
  ✓ Compose
  ✓ SpecialFunctions
  ✓ Mux
  ✓ PyCall
  ✓ WebIO
  ✓ CoupledFields
  ✓ Polynomials
  ✓ SortingLab
  ✓ MultivariateStats
  ✓ LatinHypercubeSampling
  ✓ LearnBase
  ✓ MLDatas

(nothing, nothing)

In [277]:
size(z,1)

3900

In [307]:
nfold  = 10
nobser = size(z,1) #number of observations
foldid = collect(Kfold(size(z)[1], nfold))
ytil = ones(nobser)
dtil = ones(nobser)
println("folds: " )
for i in 1:nfold
    dfit = dreg(z[foldid[i],:], d[foldid[i]])
    yfit = yreg(z[foldid[i],:], d[foldid[i]])
    dhat = predict(dfit, z[Not(foldid[i]),:])
    yhat = predict(yfit, z[Not(foldid[i]),:])
    dtil[Not(foldid[i])]   = (d[Not(foldid[i])] - dhat) 
    ytil[Not(foldid[i])]   = (y[Not(foldid[i])] - yhat)
    println(i )
end

folds: 
1
2
3
4
5
6
7
8
9
10


In [319]:
methodswith()

LoadError: UndefVarError: FixedEffect not defined

In [317]:
reg(data, @formula(ytil ~ dtil +fe(clu)))

                      Fixed Effect Model                      
Number of obs:            3900  Degrees of freedom:        196
R2:                      0.004  R2 Adjusted:            -0.049
F-Stat:                14.5105  p-value:                 0.000
R2 within:               0.004  Iterations:                  1
ytil | Estimate Std.Error t value Pr(>|t|) Lower 95% Upper 95%
--------------------------------------------------------------
dtil | 0.227822 0.0598072 3.80927    0.000  0.110563   0.34508


In [315]:
rfit = reg(data, @formula(ytil ~ dtil +fe(clu)))
coef_est = coef(rfit)[1]

0.22782154722416945

In [ ]:
function DML_for_PLM(z , d , y , dreg , yreg , nfold = 2, clu)
    nobser = size(z,1) #number of observations
    foldid = collect(Kfold(size(z)[1], nfold))
    ytil = dtil = ones(nobser)
    println("folds: " )
    for i in 1:nfolds
        dfit = dreg(z[foldid[i],:], d[folid[i]])
        yfit = yreg(z[foldid[i],:], y[folid[i]])
        dhat = predict(dfit, z[Not(foldid[i]),:])
        yhat = predict(yfit, z[Not(foldid[i]),:])
        dtil[Not(foldid[i])]   = (d[Not(foldid[i])] - dhat)
        ytil[Not(foldid[i])]   = (y[Not(foldid[i])] - yhat)
        println(i )
    end
    data = DataFrame(ytil = ytil, dtil = dtil, clu)
    rfit = reg(data, @formula(ytil ~ dtil +fe(clu)))
    coef_est = coef(rfit)[1]
    se = 
    
end

In [176]:
z = Matrix(Z);
d = D[!,1];
y = Y[!,1];
clu = rdata[!, :CountyCode];
first(DataFrame(logghomr = y,logfssl = d,CountyCode = clu ),6)

,logghomr,logfssl,CountyCode
,Float64,Float64,Int64
1,-0.134778,0.0961271,1073
2,-0.239622,0.0808094,1073
3,-0.0786772,0.0573399,1073
4,-0.331465,0.0816945,1073
5,-0.31664,0.0253655,1073
6,0.105132,-0.00677726,1073


In the following, we apply the DML approach with the differnt versions of lasso.


## Lasso

In [151]:
Pkg.add("Lasso")
using Lasso

   Resolving package versions...
  No Changes to `C:\Users\PC\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\PC\.julia\environments\v1.6\Manifest.toml`


In [219]:
Random.seed!(123)
function dreg(z,d)
    fit(LassoModel,z,d)
end
############################
function yreg(z,y)
    fit(LassoModel,z,y)
end

yreg (generic function with 1 method)

In [253]:
uwu = fit(LassoModel,z,y);

In [259]:
ddd = fill(missing, size(z,1));

In [258]:
ddd[Not(a[1])] = ()

390-element Vector{Missing}:
 missing
 missing
 missing
 missing
 missing
 missing
 missing
 missing
 missing
 missing
 missing
 missing
 missing
 ⋮
 missing
 missing
 missing
 missing
 missing
 missing
 missing
 missing
 missing
 missing
 missing
 missing

In [251]:
methodswith(LassoModel)

[1] coef(a::LassoModel, args...; kwargs...) in Lasso at C:\Users\PC\.julia\packages\StatsModels\57Kc9\src\statsmodel.jl:28
[2] confint(a::LassoModel, args...; kwargs...) in Lasso at C:\Users\PC\.julia\packages\StatsModels\57Kc9\src\statsmodel.jl:28
[3] deviance(a::LassoModel, args...; kwargs...) in Lasso at C:\Users\PC\.julia\packages\StatsModels\57Kc9\src\statsmodel.jl:28
[4] dof_residual(a::LassoModel, args...; kwargs...) in Lasso at C:\Users\PC\.julia\packages\StatsModels\57Kc9\src\statsmodel.jl:28
[5] nobs(a::LassoModel, args...; kwargs...) in Lasso at C:\Users\PC\.julia\packages\StatsModels\57Kc9\src\statsmodel.jl:28
[6] residuals(a::LassoModel, args...; kwargs...) in Lasso at C:\Users\PC\.julia\packages\StatsModels\57Kc9\src\statsmodel.jl:28
[7] response(a::LassoModel, args...; kwargs...) in Lasso at C:\Users\PC\.julia\packages\StatsModels\57Kc9\src\statsmodel.jl:28
[8] dof(a::LassoModel, args...; kwargs...) in Lasso at C:\Users\PC\.julia\packages\StatsModels\57Kc9\src\statsmodel.jl:28
[9] loglikelihood(a::LassoModel, args...; kwargs...) in Lasso at C:\Users\PC\.julia\packages\StatsModels\57Kc9\src\statsmodel.jl:28
[10] nulldeviance(a::LassoModel, args...; kwargs...) in Lasso at C:\Users\PC\.julia\packages\StatsModels\57Kc9\src\statsmodel.jl:28
[11] nullloglikelihood(a::LassoModel, args...; kwargs...) in Lasso at C:\Users\PC\.julia\packages\StatsModels\57Kc9\src\statsmodel.jl:28